## First Attempt at converting the ES-DOC Errata ISSUES and into a CSV to use to filter our CMIP6 catalogs
- MUST DOWNLOAD the command line client first (uses Python 2.7 only) 
'pip install esgissue-client'

![es-doc](../assets/es-doc.png)

### This is still a cludge

- download the web-page: https://errata.es-doc.org/static/index.html as file.html
   - navigate to page
   - select '100 / page'
   - 'Save Page As' Webpage, Complete
   - drag the .htm into this directory

In [ ]:
! /bin/grep "<tr id=" file.html > issues.txt
# For now, edit issues.txt, keeping only the lines we need  

In [ ]:
import os
import pandas as pd
from glob import glob
import qgrid

In [ ]:
filepath = 'issues.txt'
with open(filepath) as fp:
    issues = []
    line = fp.readline()
    line = fp.readline()
    while line:
        issue = line.split('"')[1]
        issues += [issue]
        command = '/usr/bin/esgissue retrieve -i $PWD/issues -d $PWD/dsets --id ' + issue
        print(command)
        os.system(command)
        break
        line = fp.readline()

In [ ]:
import json

df = pd.DataFrame(columns=['uid', 'title', 'description', 'project', 'severity', 'status','urls'])

df_list = []
for item,issue in enumerate(issues):
    file_dsets = 'dsets/dset_'+issue+'.txt'
    file_issue = 'issues/issue_'+issue+'.json'
    
    with open(file_issue) as json_file:
        dict_issue = json.load(json_file)
   
    try:
        dlist = dict_issue['urls']
    except:
        dict_issue['urls'] = []
    
    df = df.append(dict_issue,ignore_index=True)
    df_dsets = pd.read_csv(file_dsets,delim_whitespace=True,header=None)
    df_dsets = df_dsets.rename(columns={0: "file_id"}).set_index([df_dsets.index])
    df_list += [list(df_dsets.file_id.values)]

df['file_ids'] = df_list
df = df.rename(columns={"uid": "issue_uid"})
#df.file_ids.values[0]

In [ ]:
df

In [ ]:
keywords = ['issue_uid','source_id', 'experiment_id', 'member_id', 'table_id', 'variable_id', 'grid_label', 'version', 'file_id']
df_all = []
for index, row in df[:1].iterrows():
    file_id = row['file_ids']
    issue = row['issue_uid']
    dfs = pd.DataFrame(columns=keywords)
    print(index, issue, len(file_id))
    for file in file_id:
        try:
            [fill,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_version] = file.split('.')
        except:
            # one issue has two concatenated lines in the file list - go back, fix, re-run
            print('not working for ',file)
        [grid_label,version] = grid_version.split('#')
        klist = [source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,file]
        kdict = dict(zip(keywords, klist))
        dfs = dfs.append(kdict,ignore_index=True)
    df_all += [dfs]
df_expand = pd.concat(df_all,sort=False)#.drop_duplicates(subset =["file_name","version","checksum"])

In [ ]:
qgrid.show_grid(df_expand)